# Segmenting and Clustering the neighborhoods in the city of Toronto, Canada #
A peer-graded assignment in Coursera authored by Maninder Kaur


## Introduction 

In this assignment, my aim is to explore, segment, and cluster the neighborhoods in the city of Toronto.
For the Toronto neighborhood data, a [Wikipedia page](@https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) exists that has all the information that can be used to explore and cluster the neighborhoods in Toronto. The first step is to scrap the Wikipedia page and wrangle the data, then the data is cleaned, and read it into a pandas  dataframe so that it is in a structured format.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. We only process the cells that have an assigned borough. We ignore cells with a borough that is Not assigned. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup and we will use it in this project. 
Before we get the data and start exploring it, let's download all the dependencies that we will need.

### Importing libraries

In [1]:
import numpy as np #library to handle data in vectorized manner
import pandas as pd #library to convert the data in tabular form and perform data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
import json #library to handle JSON files
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and logitude values


In [2]:
import requests #library to handle requests
from pandas.io.json import json_normalize #transform JSON file into a pandas dataframe
#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#!pip install folium
import folium #map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


## Scrapping content form Wikipedia page

In [4]:
#Download data and parse it:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [5]:
#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers):
    headers[i] = str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [8]:
#Find all items and skip first one:
rows = table.findAll('tr')
rows = rows[1:len(rows)]
    
#skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows):
    rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","").replace("\/tr><td>\n","")
    
#make dataframe, expand rows and drop the old one:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n=2, expand = True)
df.drop(columns=[0],inplace=True)
df["Borough"] = df["Borough"].str.replace("\n","")
df["Postal Code"] = df["Postal Code"].str.replace("\n","")
df = df.rename(columns={"Postal Code": "Postcode"})
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
#skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)

#give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

#copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

#extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x:x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto", ""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#Combine multiple neighbourhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighbourhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i,x in df2.iterrows())
df2['Neighbourhood']=df2['Neighbourhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
df = df.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
print("Shape:", df.shape)

Shape: (103, 3)


## Get the latitude and the longitude coordinates of each neighborhood

In [17]:
# add Geo-spatial data
dfll = pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.

In [18]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


## Create a map of Toronto with neighbourhoods superimposed on top

In [21]:
#create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(loaction=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}. {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius =5,
        popup =label,
        color = 'blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto

Now we are going to start utilizing the Foursquare API to explore the neighbourhoods and segment them.


## Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'AXAKVDTUQNLAURBLATUDYVLMTDG5R55MOSL2PPK1B3LSK4SU' # your Foursquare ID
CLIENT_SECRET = 'QNG24A4WAUQ2XYE2X3NZWDFTFTIESDTSET41NBDDXOZ1KSTH' # your Foursquare Secret
VERSION = '20201220' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AXAKVDTUQNLAURBLATUDYVLMTDG5R55MOSL2PPK1B3LSK4SU
CLIENT_SECRET:QNG24A4WAUQ2XYE2X3NZWDFTFTIESDTSET41NBDDXOZ1KSTH


### Let's explore the first neighbourhood in our dataframe.
Get the Neighbourhood's name

In [28]:
toronto_data.loc[0,'Neighbourhood']

'Parkwoods'

Get the Neighbourhood's latitude and longitude values

In [29]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # nighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[0,'Longitude'] # neighbourhood longitude value
neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] #neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, neighbourhood_latitude, neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Now let's get the top 100 values that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url. 

In [31]:
LIMIT = 100 #limit of number of venues returned by Foursquare API
radius = 500 #define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXAKVDTUQNLAURBLATUDYVLMTDG5R55MOSL2PPK1B3LSK4SU&client_secret=QNG24A4WAUQ2XYE2X3NZWDFTFTIESDTSET41NBDDXOZ1KSTH&v=20201220&ll=43.7532586,-79.3296565&radius=500&limit=100'

Send the GEt request and examine the results

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fe3c6cf960a06004fbbddb0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

From the Foursquare lab in the previous module, we know that all the information is in the _items key_. before we proceed,let's borrow the **get_category_type** function from the Foursquare lab. 

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into pandas dataframe.

In [34]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) #flatten JSON

#filter columns
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


And how many venues are returned by Foursquare?

In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## Explore neighbourhoods in Toronto

Let's create a function to repeat the same process to all the neighbourhoods

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create the PI request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                neighbourhood_latitude, 
                neighbourhood_longitude, 
                radius, 
                LIMIT)
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only relevant information for each nearby venue
        venues_list.append([(
                name,
                lat,
                lng,
                v['venue']['name'],
                v['venue']['location']['lat'],
                v['venue']['location']['lng'],
                v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                            'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue',
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)
        

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [40]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park


#### Let's check the size of the resulting dataframe

In [41]:
print(toronto_venues.shape)
toronto_venues.head()

(309, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Brookbanks Park,43.751976,-79.332140,Park
4,Victoria Village,43.725882,-79.315572,KFC,43.754387,-79.333021,Fast Food Restaurant


Let's check how many venues were returned for each neighborhood

In [43]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,3,3,3,3,3,3
"Alderwood, Long Branch",3,3,3,3,3,3
"Bathurst Manor, Wilson Heights, Downsview North",3,3,3,3,3,3
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",3,3,3,3,3,3
Berczy Park,3,3,3,3,3,3
"Birch Cliff, Cliffside West",3,3,3,3,3,3
"Brockton, Parkdale Village, Exhibition Place",3,3,3,3,3,3
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 3 uniques categories.


## Anlayze each neighbourhood

In [46]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="",prefix_sep="")

#add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

#move neighbourhood column to the first column
fixed_columns =[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Fast Food Restaurant,Food & Drink Shop,Park
0,Parkwoods,0,0,1
1,Parkwoods,1,0,0
2,Parkwoods,0,1,0
3,Victoria Village,0,0,1
4,Victoria Village,1,0,0


let's examine the new dataframe

In [48]:
toronto_onehot.shape

(309, 4)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(99, 4)


,Neighbourhood,Fast Food Restaurant,Food & Drink Shop,Park
0,Agincourt,0.333333,0.333333,0.333333
1,"Alderwood, Long Branch",0.333333,0.333333,0.333333
2,"Bathurst Manor, Wilson Heights, Downsview North",0.333333,0.333333,0.333333
3,Bayview Village,0.333333,0.333333,0.333333
4,"Bedford Park, Lawrence Manor East",0.333333,0.333333,0.333333
5,Berczy Park,0.333333,0.333333,0.333333
6,"Birch Cliff, Cliffside West",0.333333,0.333333,0.333333
7,"Brockton, Parkdale Village, Exhibition Place",0.333333,0.333333,0.333333
8,"Business reply mail Processing Centre, South C...",0.333333,0.333333,0.333333
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.333333,0.333333,0.333333


#### Let's print each neighborhood along with the top 5 most common venues

In [53]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Alderwood, Long Branch----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Bathurst Manor, Wilson Heights, Downsview North----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Bayview Village----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Berczy Park----
                  venue  freq
0  Fast Food Restaurant  0.33
1     Food & Drink Shop  0.33
2                  Park  0.33


----Birch Cliff, Cliffside West----
                  venue  freq
0  Fast Foo

## Cluster neighbourhoods
Run _k-means_ to cluster the neighbourhood into 5 clusters 

In [59]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [61]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')


# check the last columns!
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's visualize the resulting clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.